In [5]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

stock_name = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{stock_name}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()


price_return = data['year1ChangePercent']


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
name_groups = list(chunks(stocks['Ticker'], 100))
name_strings = []
for i in range(0, len(name_groups)):
    name_strings.append(','.join(name_groups[i]))
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
name_strings



final_dataframe = pd.DataFrame(columns = my_columns)
for name_string in name_strings:
    batch_api_call_url_stats = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={name_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url_stats).json()
    for name in name_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            name,
            data[name]['price'],
            data[name]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns
        ),
        ignore_index = True
        )
        
        
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True, drop=True)


def portfolio_input():
    global portfolioSize
    portfolioSize = input('Enter the value of portfolio: ')
    try:
        val = float(portfolioSize)
    except ValueError:
        print("That's not a number bruh. \nTry again:")
        portfolioSize = input('Enter the value of portfolio: ')
        
        
portfolio_input()
valuePerStock = float(portfolioSize) / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    price = final_dataframe.loc[i, 'Price']
    value = math.floor(valuePerStock / price)
    final_dataframe.loc[i , 'Number of Shares to Buy'] = value
    
    

hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)


for name_string in name_strings:
    batch_api_call_url_stats = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={name_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url_stats).json()
    for name in name_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
        pd.Series(
        [
            name,
            data[name]['price'],
            'N/A',
            data[name]['stats']['year1ChangePercent'],
            'N/A',
            data[name]['stats']['month6ChangePercent'],
            'N/A',
            data[name]['stats']['month3ChangePercent'],
            'N/A',
            data[name]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
        ],
        index = hqm_columns
        ),
        ignore_index = True
        )
        
        
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
    ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        changeColumn = f'{time_period} Price Return'
        if hqm_dataframe.loc[row, changeColumn] == None:
            hqm_dataframe.loc[row, changeColumn] = 0.0

            
for row in hqm_dataframe.index:
    for time_period in time_periods:
        changeColumn = f'{time_period} Price Return'
        percentileColumn = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentileColumn] = stats.percentileofscore(hqm_dataframe[changeColumn], hqm_dataframe.loc[row, changeColumn])
        
        
        
from statistics import mean
for row in hqm_dataframe.index:
    hqm_score = mean([hqm_dataframe.loc[row, 'One-Year Return Percentile'], hqm_dataframe.loc[row, 'Six-Month Return Percentile'], hqm_dataframe.loc[row, 'Three-Month Return Percentile'], hqm_dataframe.loc[row, 'One-Month Return Percentile']])
    hqm_dataframe.loc[row, 'HQM Score'] = hqm_score
    
    

hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop=True)



position_size = float(portfolioSize) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
    
print(hqm_dataframe)

Enter the value of portfolio: 1000000
   Ticker    Price Number of Shares to Buy One-Year Price Return  \
0    TWTR    75.00                     266              0.975625   
1    VIAC    59.81                     334              0.768986   
2   DISCA    49.30                     405              0.602919   
3    PYPL   305.51                      65              1.431033   
4    AMAT   121.80                     164              0.791048   
5   DISCK    41.82                     478              0.422335   
6    ZBRA   500.19                      39              1.021212   
7    ILMN   513.31                      38              0.722602   
8    SIVB   531.30                      37              0.931054   
9    NWSA    24.90                     803              0.664804   
10    NWS    24.30                     823              0.619331   
11   APTV   159.80                     125              0.788563   
12    UAA    22.53                     887              0.306085   
13    IVZ 